In [ ]:
# Mount Google Drive (If using Colab)

from google.colab import drive
drive.mount("drive")

In [ ]:
# Install Required Libraries from PyPI

!pip install face-detection
!pip install tqdm

In [3]:
# Import Required Libraries

import os
import numpy as np
import cv2
import face_detection
from keras.applications import resnet50
from keras.models import Model
from keras.layers import Dense,Dropout,AveragePooling2D,Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tqdm

In [ ]:
# Initialize a Face Detector 
# Confidence Threshold can be Adjusted, Greater values would Detect only Clear Faces

detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

In [ ]:
# Load Pretrained ResNet50 Model (without Last few Layers) 
# Freeze all the Layers

base_network  = resnet50.ResNet50(input_shape = (224,224,3),weights='imagenet',include_top=False)
for layer in base_network.layers:
  layer.trainaibale = False

In [6]:
# Define the Face Mask Classifier Model by adding a few Layers on top of the ResNet50 Pretrained Model

classifier_network = base_network.output
classifier_network = AveragePooling2D(pool_size=(7, 7),name = "Average_Pool_Final")(classifier_network)
classifier_network = Flatten(name = "Flatten_Final")(classifier_network)
classifier_network = Dense(128, activation="relu",name = "Dense_Final")(classifier_network)
classifier_network = Dropout(0.5,name = "Dropout_Final")(classifier_network)
classifier_network = Dense(1, activation="sigmoid", name = "Sigmoid_Classifier" )(classifier_network)

mask_classifier = Model(inputs=base_network.input, outputs=classifier_network)

In [ ]:
# Set Path to the Dataset
# Faces would be extracted and placed in the specified Directory after Processing

Dataset_PATH = "drive/My Drive/Social_Distancing_with_AI/YO"
Processed_Dataset_PATH = "drive/My Drive/Social_Distancing_with_AI/Processed_Training_Data"

# Create Empty Directories
os.mkdir(Processed_Dataset_PATH)
os.mkdir(os.path.join(Processed_Dataset_PATH,"with_mask"))
os.mkdir(os.path.join(Processed_Dataset_PATH,"without_mask"))

In [ ]:
# Prepare the Data for Training 
# Extract Faces from the Dataset and Save them in the specified Directory

# There should be 2 Sub-Directories corresponding to Masked and Non-Masked Images
paths = ["with_mask","without_mask"]

loaded = 0 

for path in paths:

  curr_path = os.path.join(Dataset_PATH,path)
  
  for file_name in tqdm.notebook.tqdm(os.listdir(curr_path)):

    try:
      
      image = cv2.imread(os.path.join(curr_path,file_name))

      # Detect Faces, Crop and Save
      detections = detector.detect(image[:,:,::-1])

      for j in range(len(detections)):
    
        face = image[int(detections[j][1]):int(detections[j][3]),
                     int(detections[j][0]):int(detections[j][2])]
        
        cv2.imwrite(os.path.join(Processed_Dataset_PATH,path)+"/"+file_name,face)

        # Count Total Faces Loaded 
        loaded+=1
    
    except:
      continue

In [8]:
# Set the Hyper-Parameters

alpha = 0.00005
n_epochs = 5
mini_batch_size = 128

In [9]:
# Compile the Model

opt = Adam(learning_rate=alpha,decay = alpha/n_epochs)
mask_classifier.compile(optimizer=opt,loss="binary_crossentropy",metrics = ['accuracy'])

In [10]:
# Define a ImageDataGenerator for Real-Time Data Augmentation
# Parameters can be Tuned for controlling the Augmentation

Data_Generator = ImageDataGenerator(horizontal_flip=True,
                                    rotation_range=25,
                                    preprocessing_function=resnet50.preprocess_input)

In [ ]:
# Create a Data_Generator Instance

Train_Data_Generator = Data_Generator.flow_from_directory(Processed_Dataset_PATH,target_size=(224,224),
                                                          class_mode="binary",batch_size=mini_batch_size)

In [ ]:
# Train the Model

mask_classifier.fit(x=Train_Data_Generator,steps_per_epoch=loaded/mini_batch_size,epochs=n_epochs)

In [ ]:
# Save the Trained Weights to Disk

mask_classifier.save("drive/My Drive/Social_Distancing_with_AI/ResNet50_Classifier.h5")